# Clean movie sequel data scraped from Box Office Mojo

In [1]:
import requests
import time
import random
import pickle
import pandas as pd
import numpy as np

In [28]:
with open('raw_movie_data.pickle','rb') as read_file:
    df = pickle.load(read_file)

In [ ]:
# check for duplicate entries - in data cleaning notebook next
# add a dummy variable for part of meta-franchise yes or no
# change '0s' in theateres to numpy NaN
# remove re-releases, IMAX, 3D - basically any movie with parentheses in it

# after removing dupes and re-releases/IMAX/etc., only then sort and get stats based on prior movie in the franchise

## Check for re-releases, IMAX, 3D - movies with parentheses in the name

These movies are not truly new sequels, but rather re-releases or special releases of other sequels after the first release. I'm only interested in new sequels or remakes. 

In [29]:
mask = (df.movie_title.str.contains("\([^\)]*\)"))
df[mask]

,franchise_name,movie_title,rank,studio,adjusted_domestic_gross,release_date,theaters,movie_webpage
6,Agatha Christie,Murder on the Orient Express (2017),1,Fox,102608700,2017-11-10,3354,http://www.boxofficemojo.com/franchises/movies...
10,Alice in Wonderland,Alice in Wonderland (2010),1,BV,385654200,2010-03-05,3739,http://www.boxofficemojo.com/franchises/movies...
34,Amityville,The Amityville Horror (2005),2,MGM,93219600,2005-04-15,3323,http://www.boxofficemojo.com/franchises/movies...
44,Arthur,Arthur (2011),2,WB,37543900,2011-04-08,3276,http://www.boxofficemojo.com/franchises/movies...
53,Avatar,Avatar: Special Edition(Re-release),2,Fox,12750400,2010-08-27,812,http://www.boxofficemojo.com/franchises/movies...
71,Barbershop,Beauty Shop((spin-off)),4,MGM,51946700,2005-03-30,2659,http://www.boxofficemojo.com/franchises/movies...
138,Conan,Conan the Barbarian (2011),3,LGF,24568300,2011-08-19,3015,http://www.boxofficemojo.com/franchises/movies...
151,The Dark Knight Trilogy,The Dark Knight (2012 re-release)(Re-release),4,WB,1781500,2012-07-19,0,http://www.boxofficemojo.com/franchises/movies...
152,The Dark Knight Trilogy,Batman Begins (2012 re-release)(Re-release),5,WB,1776300,2012-07-19,0,http://www.boxofficemojo.com/franchises/movies...
153,Dark Universe,The Mummy (2017),1,Uni.,82129900,2017-06-09,4035,http://www.boxofficemojo.com/franchises/movies...


Some movies are remakes and have the new year of the remake in parentheses - for example, the Alice in Wonderland 2010 remake is called "Alice in Wonderland (2010)". One franchise actually has movies with a word in parentheses. These are data points we are interested in, so we'll have to remove the re-releases, IMAX and 3D movies separately so as to keep the remakes in the dataset. 

In [40]:
mask = (df.movie_title.str.contains("\(Re-release\)"))
print(df[mask].movie_title)
df_no_rerelease = df[~mask]
len(df_no_rerelease)

53                   Avatar: Special Edition(Re-release)
151        The Dark Knight (2012 re-release)(Re-release)
152          Batman Begins (2012 re-release)(Re-release)
218              The Exorcist Director's Cut(Re-release)
276                  Ghostbusters (Re-issue)(Re-release)
277    Ghostbusters (30th Anniversary re-release)(Re-...
284                 The Godfather (Re-issue)(Re-release)
340    Herbie Goes to Monte Carlo (Re-issue)(Re-release)
341           Herbie Goes Bananas (Re-issue)(Re-release)
381    Raiders of the Lost Ark (Re-issue) (1982)(Re-r...
382    Raiders of the Lost Ark (Re-issue) (1983)(Re-r...
447                   Jaws 2 (1980 re-issue)(Re-release)
683                   Poltergeist (re-issue)(Re-release)
687                       Porky's (Re-issue)(Re-release)
745                     Rocky III (Re-issue)(Re-release)
835              Star Wars (Special Edition)(Re-release)
837    The Empire Strikes Back (Special Edition)(Re-r...
838     Return of the Jedi (Spe

950

Now search for IMAX releases.

In [41]:
mask = (df_no_rerelease.movie_title.str.contains("IMAX"))
print(df_no_rerelease[mask].movie_title)
df_no_rerelease_IMAX = df_no_rerelease[~mask]
len(df_no_rerelease_IMAX)

326    Harry Potter IMAX Marathon
406    Harry Potter IMAX Marathon
Name: movie_title, dtype: object


948

Now search for and check 3D releases to see if they are considered separately

In [43]:
mask = (df_no_rerelease_IMAX.movie_title.str.contains("3D"))
print(df_no_rerelease_IMAX[mask].movie_title)

316    A Very Harold & Kumar 3D Christmas
760                                Saw 3D
810                Spy Kids 3D: Game Over
882                     Texas Chainsaw 3D
896          Toy Story / Toy Story 2 (3D)
Name: movie_title, dtype: object


In [47]:
mask = df_no_rerelease_IMAX.movie_title.str.contains("Toy Story / Toy Story 2")
print(df_no_rerelease_IMAX[mask].movie_title)
df_no_rerelease_IMAX_3D = df_no_rerelease_IMAX[~mask]
len(df_no_rerelease_IMAX_3D)

896    Toy Story / Toy Story 2 (3D)
Name: movie_title, dtype: object


947

In [50]:
print(str(len(df) - len(df_no_rerelease_IMAX_3D)) + ' movies removed as re-releases, either as IMAX, 3D or standard re-releases.')

27 movies removed as re-releases, either as IMAX, 3D or standard re-releases.


## Find movies that are duplicated in the dataset under multiple franchises.  

Sometimes a movie is classified under multiple franchises. This will skew the prediction, as we are basing modeling of current sequel box office on how the last movie did in the series. If Deadpool 2 belongs to both the X-Men franchise and the Deadpool franchise, then the exact Y value will be represented twice with two separate sets of X values (one set using Deadpool 1 as the comparator movie, while the other will use the most recent X-Men movie, which could be X-Men Evolution, or Logan).  
  
We will add a feature to classify if a movie originally appeared under multiple franchises (0 or 1), and then will remove the movie from the larger franchise in favor of the smaller, more specific franchise.

In [80]:
df_duplicates = df_no_rerelease_IMAX_3D.groupby(by=['movie_title', 'adjusted_domestic_gross'])['franchise_name'].agg('count').sort_values(ascending=False).reset_index()

In [82]:
df_duplicates.columns = ['movie_title', 'adjusted_domestic_gross', 'n_appearances']

In [86]:
mask = (df_duplicates.n_appearances > 1)
df_duplicates = df_duplicates[mask]
df_duplicates

,movie_title,adjusted_domestic_gross,n_appearances
0,Batman v Superman: Dawn of Justice,350467400,3
1,Spider-Man: Homecoming,342750600,3
2,The Lord of the Rings: The Fellowship of the Ring,498942900,2
3,Logan,234205600,2
4,Annabelle,93005800,2
5,Deadpool,387403700,2
6,Thor,205797400,2
7,Thor: Ragnarok,314382100,2
8,Thor: The Dark World,226626300,2
9,Dawn of the Planet of the Apes,236385100,2


#### Create a dataframe of the franchises that are in the above data frame so I know which ones need to be cleaned 

* The 20 movies within the Marvel Universe tag can be removed, because they have their own series listed elsewhere in the dataset.  
* The Middle Earth franchise can also be removed.  
* Alien vs. Predator can be considered a new franchise with 2 movies. 
* Minions can be contained only in the Minions franchise, not Despicable Me
* Star Wars Anthology can be removed
* Batman v Superman movie was removed
* DC Universe franchise can be removed completely - contained in other franchises
* Wolverine and Deadpool movies can be removed from the X-Men franchise
* The Dark Knight can be removed from Batman  
 Next: Godzilla, King Kong, Alice in Wonderland, Freddy vs Jason, Harry Potter, Malificent, Annabelle

## Sort data by release date 

In [ ]:
df = df.sort_values(by=['franchise_name', 'release_date'])

In [ ]:
df[['prior_movie_studio', 'prior_movie_release_date']] = df.groupby(['franchise_name'])['studio','release_date'].transform(lambda grp: grp.shift(1))

In [ ]:
df['prior_movie_adj_domestic_gross'] = df.groupby(['franchise_name'])['adjusted_domestic_gross'].transform(lambda grp: grp.shift(1))

In [ ]:
df.head()

In [ ]:
df['time_since_prior_movie'] = df.release_date - df.prior_movie_release_date

In [ ]:
# Ask the teachers if there's a better way to do this
franchise_start_date = df.groupby(by='franchise_name')['release_date'].min().reset_index()
franchise_start_date.columns = ['franchise_name', 'first_movie_release_date']
df = pd.merge(df,franchise_start_date,on='franchise_name')

In [ ]:
df['time_since_first_movie'] = df.release_date - df.first_movie_release_date

In [ ]:
# get first of each domestic gross for the first movie of each franchise, add that as a column, then divide current movie's gross by that
first_movie_adj_domestic_gross = pd.DataFrame(df.groupby(by=['franchise_name'])['adjusted_domestic_gross'].first())

In [ ]:
first_movie_adj_domestic_gross.head()

In [ ]:
first_movie_adj_domestic_gross.columns = ['first_movie_adj_domestic_gross']

In [ ]:
first_movie_adj_domestic_gross.head()

In [ ]:
df = pd.merge(df,first_movie_adj_domestic_gross,on='franchise_name')

In [ ]:
df['percent_gross_change_vs_first_movie'] = df.adjusted_domestic_gross/df.first_movie_adj_domestic_gross

In [ ]:
df['ln_adj_domestic_gross'] = np.log(df.adjusted_domestic_gross)

In [ ]:
df['ln_prior_movie_adj_domestic_gross'] = np.log(df.prior_movie_adj_domestic_gross)

In [ ]:
df['ln_first_movie_adj_domestic_gross'] = np.log(df.first_movie_adj_domestic_gross)

In [ ]:
# see if a change in the studio from the last movie to this one impacts the outcome
test = df.head()
df.studio_changed_vs_prior_movie = np.where(df.studio == df.prior_movie_studio, 0, 1)
df.studio_changed_vs_prior_movie[df.order == 1] = np.NaN

In [ ]:
df.head()

In [ ]:
df.head()

In [ ]:
sns.heatmap(df.corr(), cmap="Blues")

In [ ]:
df.corr()

### Remove all first movies in franchises - look only at 2nd and onward

In [ ]:
mask = (df.order > 1)
sequel_df = df[mask]

In [ ]:
sequel_df.tail()

In [ ]:
sns.heatmap(sequel_df.corr(), cmap="Blues")

In [ ]:
mask = (df.order > 2)
third_on_df = df[mask] 

In [ ]:
sns.heatmap(third_on_df.corr(), cmap="Blues")

### Order of movie in the franchise
'rank' from BoxOfficeMojo is the rank by box office, not release order

In [ ]:
df.head()

In [ ]:
name = df.franchise_name[0]
counter = 1
rank_list = []
for i in df.franchise_name: 
    if i == name:
        rank_list.append(counter)
        counter+=1
    else:
        counter = 1
        rank_list.append(counter)
        counter+=1
        name = i
df['order'] = np.array(rank_list)

In [ ]:
df.columns

## Quick viz 

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
df['time_since_prior_movie_int'] = df.time_since_prior_movie.dt.days

In [ ]:
df['time_since_first_movie_int'] = df.time_since_first_movie.dt.days

In [ ]:
x = df.time_since_prior_movie_int
y = df.adjusted_domestic_gross

plt.scatter(x,y, alpha = 0.5);
plt.xlim(0,1500)

In [ ]:
mask = (df.order > 1)
x = df.order[mask]
y = df.adjusted_domestic_gross[mask]

plt.scatter(x,y, alpha = 0.5);
plt.xlim(0,10)

In [ ]:
mask = (df.order > 1)
x = df.order[mask]
y = df.percent_gross_change_vs_first_movie[mask]

plt.scatter(x,y, alpha = 0.5);
plt.xlim(0,10)
plt.ylim(0,2)

In [ ]:
mask = (df.order > 1)
x = df.order#[mask]
y = df.ln_adj_domestic_gross#[mask]

plt.scatter(x,y, alpha = 0.5);
plt.xlim(0,10)
# plt.ylim(0,2)

In [ ]:
mask = (df.time_since_first_movie_int > 0)
x = df.time_since_first_movie_int[mask]
y = df.adjusted_domestic_gross[mask]

plt.scatter(x,y, alpha = 0.5);
plt.xlim(0,10000)

In [ ]:
df.groupby(by='time_since_prior_movie_int').size().sort_values(ascending=False).head()

In [ ]:
x = df.time_since_prior_movie_int
y = df.percent_gross_change_vs_first_movie

plt.scatter(x,y, alpha = 0.5);
plt.ylim(0,2)
plt.xlim(0,1200)

In [ ]:
x = df.time_since_first_movie_int
y = df.percent_gross_change_vs_first_movie

plt.scatter(x,y, alpha = 0.5);
plt.ylim(0,2)
plt.xlim(0,2000)

In [ ]:
x = df.prior_movie_adj_domestic_gross
y = df.adjusted_domestic_gross

plt.scatter(x,y, alpha = 0.5);
#plt.ylim(0,2)
#plt.xlim(0,1000000)

In [ ]:
x = df.ln_prior_movie_adj_domestic_gross
y = df.ln_adj_domestic_gross

plt.scatter(x,y, alpha = 0.5);
plt.ylim(14,21)
plt.xlim(14,21)
plt.xlabel('prior movie domestic gross\n(ln $, inflation adj)')
plt.ylabel('next movie domestic gross\n(ln $, inflation adj)')

In [ ]:
df.groupby('')

In [ ]:
df.head()

# First pass model

In [ ]:
df.columns

In [ ]:
y, X = patsy.dmatrices('ln_adj_domestic_gross ~ ln_prior_movie_adj_domestic_gross + ln_first_movie_adj_domestic_gross + time_since_prior_movie_int + time_since_first_movie_int + order + studio_changed_vs_prior_movie', data=sequel_df, return_type="dataframe")

model_a = sm.OLS(y, X)
fit_a = model_a.fit()
fit_a.summary()

In [ ]:
fit_a.resid.plot(style='o', figsize=(12,8));

In [ ]:
plt.hist(fit_a.resid)

In [ ]:
df.shape

In [ ]:
y, X = patsy.dmatrices('ln_adj_domestic_gross ~ ln_prior_movie_adj_domestic_gross + ln_first_movie_adj_domestic_gross + order + studio_changed_vs_prior_movie + time_since_prior_movie_int', data=sequel_df, return_type="dataframe")

model_b = sm.OLS(y, X)
fit_b = model_b.fit()
fit_b.summary()

In [ ]:
fit_b.resid.plot(style='o', figsize=(12,8));

In [ ]:
plt.hist(fit_b.resid)

In [ ]:
import scipy

In [ ]:
scipy.stats.normaltest(fit_b.resid)

In [ ]:
np.random.seed(28041990)

# Code for The Numbers website for direct to video sequels

Below is getting data from The Numbers - include direct-to-video movies which is interesting, but they don't have sales data for all of those direct to video releases so it's tougher to assess. It might be useful down the line.

In [ ]:
"""
url = 'https://www.the-numbers.com/movies/franchises'
response = requests.get(url)
response.status_code
# print(response.text)
page = response.text
soup = BeautifulSoup(page,"lxml")
# soup.find_all('table')
# soup.find('table').find_all('tr')[1:]

movie_count = 0
single = 0
franchises = 0
for link in soup.find('table').find_all('tr')[1:]:
    print(link.find('a')['href']) # put .text instead of ['href'] for the name of the franchise
    n = int(link.find_all('td')[1].text)
    print(link.find_all('td')[1].text) # print the next column - the number of movies
    movie_count += int(link.find_all('td')[1].text)
    if n == 1:
        single += 1
    franchises += 1
print(movie_count)
print(franchises)
print(single)
"""